In [15]:
import pandas as pd
from os import scandir, rename
from os.path import join
from tqdm.notebook import tqdm
from IPython.display import display
dataset = "scsyst"

for split in ['val_train','test']: # Find all final epoch results CSVs
    exps = []
    for f in scandir(dataset):
        for f1 in scandir(f.path):
            #if "jto" in f1.path:
                exps.append(join(f1.path,f"{split}.csv"))
    results  = pd.DataFrame()
    # Create 
    for f in tqdm(exps):
        csv = pd.read_csv(f)
        method = f.split("/")[1]
        data = f.split("/")[2]
        hidden_dim = data.split("_")[0]
        seed = data.split("_")[1]  
        #print(f,method,corr)
        csv['method'] = method
        csv['hidden_dim'] = hidden_dim
        csv['seed'] = seed

        results = pd.concat([results, csv])
        #print(results.columns)
    results.to_csv(f'{dataset}_{split}.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [3]:
!pwd

/media/alain/Data/Tesis/scsyst


In [5]:
!dir

args.py								 models.py
circle_[0.\ \ \ \ \ \ \ \ \ 0.\ \ \ \ \ \ \ \ \ 0.33333334].png  __pycache__
circle_[[0.33333334\ 0.33333334\ 0.6666667\ ]].png		 results
circle_[[0.6666667\ 0.\ \ \ \ \ \ \ \ 1.\ \ \ \ \ \ \ ]].png	 run_expt.py
Compile_Results.ipynb						 SCSyst.ipynb
datasets.py							 train.py
feats								 utils.py
generate_dataset.ipynb


In [6]:
results

,loss,acc,epoch,method,hidden_dim,seed
0,76.033211,0.000000,1,erm,100,939042955
1,75.647766,0.000000,2,erm,100,939042955
2,75.259392,0.000000,3,erm,100,939042955
3,74.869286,0.000000,4,erm,100,939042955
4,74.475815,0.000000,5,erm,100,939042955
...,...,...,...,...,...,...
995,3.491383,30.000001,996,erm,100,127978094
996,3.488015,30.000001,997,erm,100,127978094
997,3.481783,30.000001,998,erm,100,127978094
998,3.479032,30.000001,999,erm,100,127978094


In [12]:
a = {50: 0.0, 625: 0.25, 75:0.5, 875: 0.75, 95:0.9}
def format_perc(perc):
    return '{:.2f}\%'.format(perc)
def format_float(perc):
    return '{:.2f}'.format(perc)
splits = ['train','val','test']

'''
- 'means' = overall results per method, get max value from each seed, 
then average results by seed. For results table.
- 'all' = raw results from csvs
- 'seed' = best result for seed
- 'epoch' = results averaged by seed at the epoch level. For plotting.
'''
def create_results_table(dataset, split, metric="worst_acc"):
    results = pd.read_csv(f'{dataset}_{split}.csv')
    epoch_df = results.groupby(['method',"epoch"]).agg(
        {'loss': 'mean','acc': 'mean'}
    ).reset_index()
    grouped_df = results.groupby(['method','seed']).agg(
        {'loss': 'min', 'acc': 'max'}
    ).reset_index()
    means_df = grouped_df.groupby(['method']).agg(
        {'loss':'mean','acc':'mean'}
    ).reset_index()
    for m in ['loss','acc']:
        means_df[m] = means_df[m]
        means_df[f'{m}_std'] = grouped_df.groupby(['method'])[m].std().reset_index()[m]
        means_df['count'] = grouped_df.groupby(['method'])[m].count().reset_index()[m]
        means_df[m] = means_df[m].apply(format_perc)
        means_df[f'{m}_std'] = means_df[f'{m}_std'].apply(format_float)
        means_df[f'final_{m}'] = (means_df[m]).astype(str) + " $\pm$ " + means_df[f'{m}_std'].astype(str) + " (" + means_df['count'].astype(str) + ")"
    
    return {'means': means_df, 'seed': grouped_df, 'all': results, 'epoch': epoch_df}

def create_final_table(df, metric="avg_acc"):
    final_table = df.pivot(index='method',
                           columns='corr',
                           values=([f"final_{metric}"])).reset_index()

    return final_table

In [20]:
r = create_results_table("scsyst","test")['means']

In [21]:
create_final_table(r, metric="acc")

KeyError: 'corr'